In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import torch
import torch.nn as nn

In [2]:
path = "PYTORCH_NOTEBOOKS/Data"

In [3]:
df = pd.read_csv(path + '/income.csv')

In [4]:
df.head()

,age,sex,education,education-num,marital-status,workclass,occupation,hours-per-week,income,label
0,27,Male,HS-grad,9,Never-married,Private,Craft-repair,40,<=50K,0
1,47,Male,Masters,14,Married,Local-gov,Exec-managerial,50,>50K,1
2,59,Male,HS-grad,9,Divorced,Self-emp,Prof-specialty,20,<=50K,0
3,38,Female,Prof-school,15,Never-married,Federal-gov,Prof-specialty,57,>50K,1
4,64,Female,11th,7,Widowed,Private,Farming-fishing,40,<=50K,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
age               30000 non-null int64
sex               30000 non-null object
education         30000 non-null object
education-num     30000 non-null int64
marital-status    30000 non-null object
workclass         30000 non-null object
occupation        30000 non-null object
hours-per-week    30000 non-null int64
income            30000 non-null object
label             30000 non-null int64
dtypes: int64(4), object(6)
memory usage: 2.3+ MB


In [6]:
df.columns

Index(['age', 'sex', 'education', 'education-num', 'marital-status',
       'workclass', 'occupation', 'hours-per-week', 'income', 'label'],
      dtype='object')

In [7]:
cat_cols = ['sex','education','marital-status','workclass','occupation']
cont_cols = ['age','hours-per-week']
y = ['label']

In [8]:
for col in cat_cols:
    df[col] = df[col].astype('category')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
age               30000 non-null int64
sex               30000 non-null category
education         30000 non-null category
education-num     30000 non-null int64
marital-status    30000 non-null category
workclass         30000 non-null category
occupation        30000 non-null category
hours-per-week    30000 non-null int64
income            30000 non-null object
label             30000 non-null int64
dtypes: category(5), int64(4), object(1)
memory usage: 1.3+ MB


In [10]:
cat_size = ([len(df[col].cat.categories) for col in cat_cols])

In [11]:
cat_size

[2, 14, 6, 5, 12]

In [12]:
emb_size = [(size,min(size,(size+1)//2)) for size in cat_size]

In [13]:
emb_size

[(2, 1), (14, 7), (6, 3), (5, 3), (12, 6)]

In [14]:
cats = np.stack([df[col].cat.codes.values for col in cat_cols],1)

In [15]:
cats

array([[ 1, 10,  3,  2,  1],
       [ 1, 11,  1,  1,  2],
       [ 1, 10,  0,  3,  7],
       ...,
       [ 1, 12,  1,  2,  7],
       [ 0, 13,  3,  2,  0],
       [ 1,  6,  1,  3,  2]], dtype=int8)

In [16]:
cont = np.stack([df[col].values for col in cont_cols],1)

In [17]:
cont

array([[27, 40],
       [47, 50],
       [59, 20],
       ...,
       [47, 55],
       [32, 40],
       [33, 60]], dtype=int64)

In [18]:
cats = torch.tensor(cats,dtype = torch.int64)
cont = torch.tensor(cont,dtype = torch.float)
y_val = torch.tensor(df[y].values).flatten()

In [19]:
cont.shape[1]

2

In [47]:
class TabularModel1(nn.Module):
    
    def __init__(self,emb_size,n_cont,out_size,layers,p= 0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(ni,nf) for ni,nf in emb_size])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum((nf for ni,nf in emb_size))
        n_in=  n_cont + n_emb
        layerlist = []
        for i in layers:
            layerlist.append(nn.Linear(n_in,i))
            layerlist.append(nn.ReLU(inplace = True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_size))
        self.layers = nn.Sequential(*layerlist)
    def forward(self,x_cat,x_cont):
        embedingz = []
        for i,e in enumerate(self.embeds):
            embedingz.append(e(x_cat[:,i]))
        x = torch.cat(embedingz,1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x,x_cont],1)
        x = self.layers(x)
        return x

In [48]:

model = TabularModel1(emb_size,cont.shape[1],2,[200,100])

In [49]:
model

TabularModel1(
  (embeds): ModuleList(
    (0): Embedding(2, 1)
    (1): Embedding(14, 7)
    (2): Embedding(6, 3)
    (3): Embedding(5, 3)
    (4): Embedding(12, 6)
  )
  (emb_drop): Dropout(p=0.5, inplace=False)
  (bn_cont): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=22, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [50]:
len(df)

30000

In [51]:
batch_size = 30000
test_size = int(batch_size*0.2)

In [52]:
cat_train = cats[:batch_size-test_size]
cat_test = cats[batch_size-test_size:batch_size]
cont_train = cont[:batch_size-test_size]
cont_test = cont[batch_size-test_size:batch_size]


In [53]:
y_train = y_val[:batch_size-test_size]
y_test = y_val[batch_size-test_size:batch_size]

In [54]:
criterion=  nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [55]:
losses = []
epochs = 300
for i in range(epochs):
    i+=1
    y_pred = model(cat_train,cont_train)
    loss = criterion(y_pred,y_train)
    losses.append(loss)
    
    if i%25==1:
        print(f"{i} epoch have loss:{loss}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

1 epoch have loss:0.9015924334526062
26 epoch have loss:0.3823983073234558
51 epoch have loss:0.32577717304229736
76 epoch have loss:0.30450063943862915
101 epoch have loss:0.2928931713104248
126 epoch have loss:0.2857554852962494
151 epoch have loss:0.28509756922721863
176 epoch have loss:0.28050172328948975
201 epoch have loss:0.2798992097377777
226 epoch have loss:0.27672234177589417
251 epoch have loss:0.2739979922771454
276 epoch have loss:0.27056998014450073


In [56]:
with torch.no_grad():
    y_eval = model(cat_test,cont_test)
    loss = criterion(y_eval,y_test)

In [57]:
loss

tensor(0.2816)

In [64]:
# DON'T WRITE HERE
rows = len(y_test)
correct = 0

print(f'{"MODEL OUTPUT":26} ARGMAX  Y_TEST')

for i in range(rows):
    print(f'{str(y_val[i]):26} {y_val[i].argmax().item():^7}{y_test[i]:^7}')

    if y_val[i].argmax().item() == y_test[i]:
        correct += 1

print(f'\n{correct} out of {rows} = {100*correct/rows:.2f}% correct')

MODEL OUTPUT               ARGMAX  Y_TEST
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0   

tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0   

tensor(0)                     0      0   
tensor(1)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0   

tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(1)                     0      0   
tensor(1)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0   

tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(1)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0   

...

tensor(1)                     0      0   
tensor(1)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0   

tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0   

tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0   

tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(1)                     0      1   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0   

tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0      0   
tensor(1)                     0      1   
tensor(0)                     0      0   
tensor(1)                     0      1   
tensor(0)                     0      1   
tensor(0)                     0      0   
tensor(0)                     0      0   
tensor(1)                     0      0   
tensor(0)                     0      0   
tensor(0)                     0      1   
tensor(1)                     0   